# Sample data preparation: SQuAD question answering

> *This notebook has been tested in the Python 3 kernel of SageMaker Studio JupyterLab (Distribution v1.6)*

This notebook records the steps we took to transform and prepare a small extract of the [Stanford Question Answering Dataset (SQuAD) v2.0](https://rajpurkar.github.io/SQuAD-explorer/) for use with Foundation Model Evaluation Jobs [in Amazon Bedrock](https://docs.aws.amazon.com/bedrock/latest/userguide/model-evaluation.html) and [in Amazon SageMaker](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-foundation-model-evaluate.html).

You don't need to re-run it (unless you want to explore further): The output files are saved in this folder of the repository.

## Imports and setup

This notebook optionally uses [OpenPyXL](https://openpyxl.readthedocs.io/en/stable/tutorial.html) to demonstrate working with `.xlsx` files from Python. This library isn't installed in the SageMaker Studio Python kernel by default, so you'll need to install it to run that section:

In [1]:
%pip install openpyxl

  Using cached openpyxl-3.1.2-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-1.1.0-py3-none-any.whl.metadata (1.8 kB)
Using cached openpyxl-3.1.2-py2.py3-none-any.whl (249 kB)
Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Python Built-Ins:
import csv
import json
import os
from string import Template
from typing import Dict, List

# External Dependencies:
import openpyxl  # Utilities for working with .xls[x] spreadsheets
import pandas as pd  # Utilities for working with dataframes (tabular data)
from tqdm.notebook import tqdm  # Progress bars

os.makedirs("raw/squad", exist_ok=True)  # Raw source data
os.makedirs("question-answering", exist_ok=True)  # Transformed output dataset

## Fetch and pre-process raw SQuAD data

This sample uses (a small subset of) the [Stanford Question Answering Dataset (SQuAD) v2.0](https://rajpurkar.github.io/SQuAD-explorer/), development partition. SQuAD (also available [on Hugging Face Datasets](https://huggingface.co/datasets/rajpurkar/squad_v2)) is distributed under the CC BY-SA 4.0 license.

First we'll fetch the raw JSON dataset from the original source:

In [3]:
!wget -O raw/squad/squad-dev-v2.0.json https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json

--2024-04-29 17:19:29--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.111.153, 185.199.110.153, 185.199.109.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.111.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4370528 (4.2M) [application/json]
Saving to: ‘raw/squad/squad-dev-v2.0.json’

raw/squad/squad-dev 100%[===================>]   4.17M  --.-KB/s    in 0.01s   

2024-04-29 17:19:29 (301 MB/s) - ‘raw/squad/squad-dev-v2.0.json’ saved [4370528/4370528]



SQuAD is based on Wikipedia articles and demonstrates **extractive question answering in context**: Each paragraph of an article is linked to a set of questions that paragraph directly answers, and links to which exact span(s) of text answers each question.

For our case we'd like to demonstrate a **RAG-like** use-case, so we'll concatenate all the paragraphs of each article together into one overall "document", and sample just one question per article.

We'll output the result into a [JSON-Lines](https://jsonlines.org/) format where each line/record contains:

```json
{
    "doc": "...", // Text of the article
    "doc_id": "...", // Unique ID (title) of the article
    "question": "...", // Text of the question
    "question_id": "...", // Unique SQuAD ID of the question
    "answers": ["...", "..."], // Set of (potentially multiple) reference answer texts
}
```

In [4]:
n_docs = 0
n_qs = 0
with open("raw/squad/squad-dev-v2.0.json") as fin:
    raw = json.load(fin)
    with open("question-answering/qa.manifest.jsonl", "w") as fout:
        for doc_data in tqdm(raw["data"]):
            n_docs += 1
            doc_text = "\n\n".join(para["context"] for para in doc_data["paragraphs"])
            for para in doc_data["paragraphs"]:
                for qa in para["qas"]:
                    if qa["is_impossible"]:
                        continue
                    if n_qs > 0:
                        fout.write("\n")
                    # We use dict over 'set' here to keep deterministic order:
                    answers = {ans["text"]: None for ans in qa["answers"]}
                    n_qs += 1
                    fout.write(
                        json.dumps({
                            "doc": doc_text,
                            "doc_id": doc_data["title"],
                            "question": qa["question"],
                            "question_id": qa["id"],
                            "answers": list(answers.keys())
                        })
                    )
                    break
                break

print(f"Got {n_qs} questions from {n_docs} documents")

  0%|          | 0/35 [00:00<?, ?it/s]

Got 35 questions from 35 documents


## Converting between JSON and XLSX

JSON/JSON-Lines files are well-structured to use from code, but business users would typically be more comfortable building and maintaining datasets in spreadsheets to avoid having to add extra notation to their example texts.

As shown in the snippet below, it's pretty straightforward to convert between JSON-Lines and **CSV** - which may be sufficient for some use-cases:

In [ ]:
with open("question-answering/qa.manifest.jsonl") as fin:
    with open("question-answering/qa.csv", "w") as fout:
        writer = csv.writer(fout)
        writer.writerow(["Question", "Reference Answer(s)", "Source Doc ID"])
        for line in fin:
            datum = json.loads(line)
            writer.writerow([datum["question"], "<OR>".join(datum["answers"]), datum["doc_id"]])

...But new users often experience issues (for example [with large numbers](https://answers.microsoft.com/en-us/msoffice/forum/all/microsoft-excel-corrupts-data-in-csv-files/8337e85c-b1f5-4e99-9ca4-1ab51ae2984e) or [non-ASCII characters like smart-quotes](https://answers.microsoft.com/en-us/msoffice/forum/all/csv-opening-in-excel-corrupts-the-text/18b997a2-a6fb-4d4c-8ffd-a40176961421)) when working in CSV data in common spreadsheet applications like Excel.

We can simplify the editing experience by exporting directly into native Excel formats, using the open-source [openpyxl](https://openpyxl.readthedocs.io/en/stable/) library as shown below:

In [ ]:
with open("question-answering/qa.manifest.jsonl") as fin:
    wb = openpyxl.Workbook()
    ws = wb.active
    ws.title = "Test Cases"

    # Push the data:
    ws["A1"] = "Question"
    ws["B1"] = "Reference Answer(s)"
    ws["C1"] = "Source Doc ID"
    for ix, line in enumerate(fin):
        datum = json.loads(line)
        ws[f"A{ix+2}"] = datum["question"]
        ws[f"B{ix+2}"] = "<OR>".join(datum["answers"])
        ws[f"C{ix+2}"] = datum["doc_id"]

    # Configure the Table (optional but nice for recording the semantics of the data)
    tab = openpyxl.worksheet.table.Table(displayName="tbl_annotations", ref=f"A1:C{ix+2}")
    tab.tableStyleInfo = openpyxl.worksheet.table.TableStyleInfo(
        name="TableStyleLight1",
        showFirstColumn=False,
        showLastColumn=False,
        showColumnStripes=False,
        showRowStripes=True,
    )
    ws.add_table(tab)

    # Save the file:
    wb.save("question-answering/qa.xlsx")

Since XLSX is a more open-ended format, converting back to JSON(L) requires deciding how strict or flexible you'd like to be in parsing out your desired structure:

In [7]:
def read_excel_annotations(filepath: str) -> List[Dict]:
    annotations_wb = openpyxl.load_workbook(filepath)
    annotations_ws = annotations_wb.active  # Assume default active worksheet ('Sheet1' or etc) is the target

    # You could spend a lot of time building fancy logic to auto-detect which worksheet, columns,
    # rows to read data from... But 'tables' are a nice semantic way to structure data in
    # spreadsheets so here we'll use that. See also:
    # https://support.microsoft.com/en-us/office/overview-of-excel-tables-7ab0bb7d-3a9e-4b56-a3c9-6c94334e492c
    # https://openpyxl.readthedocs.io/en/stable/worksheet_tables.html
    try:
        # Just take the first 'Table' to appear in the worksheet's .tables name:table dictionary:
        annotations_tbl = next(val for val in annotations_ws.tables.values())
        # Check the headers are as we expect:
        expected_header_substrs = [
            "Question", "Reference Answer", "Source",
        ]
        for ixcol, substr in enumerate(expected_header_substrs):
            assert substr in annotations_tbl.column_names[ixcol], \
                "Expected column {} of table {} to have '{}' in title. Got '{}'".format(
                    ixcol, annotations_tbl.name, substr, annotations_tbl.column_names[ixcol]
                )
        # Fetch the data cells from the table:
        start_row_offset = annotations_tbl.headerRowCount
        end_row_offset = -annotations_tbl.totalsRowCount if annotations_tbl.totalsRowCount else None
        data_cells = annotations_ws[annotations_tbl.ref][start_row_offset:end_row_offset]
        results = []
        for ixrow, row_cells in enumerate(data_cells):
            # Skip blank lines:
            if all(not cell.value for cell in row_cells):
                continue
            # Append record:
            results.append({
                "question": row_cells[0].value,
                "answers": row_cells[1].value,
                "doc_id": row_cells[2].value,
            })
        return results
    except Exception as err:
        raise ValueError(
            "Failed to read annotation data from spreadsheet - maybe you need to update the logic or "
            "make your spreadsheet consistent with it?"
        ) from err


annotations = read_excel_annotations("question-answering/qa.xlsx")
for ix, ann in enumerate(annotations):
    print(ann)
    if ix >= 2:
        print("...")
        break

{'question': 'In what country is Normandy located?', 'answers': 'France', 'doc_id': 'Normans'}
{'question': 'What branch of theoretical computer science deals with broadly classifying computational problems by difficulty and class of relationship?', 'answers': 'Computational complexity theory', 'doc_id': 'Computational_complexity_theory'}
{'question': 'What is Southern California often abbreviated as?', 'answers': 'SoCal', 'doc_id': 'Southern_California'}
...


## From dataset and prompt template to model inputs

To answer questions from source data with our Large Language Model, we'll use a **prompt template** to combine:

1. General persona-setting guidance about how the bot should behave and respond to the question (sometimes called a "system message" or "system context")
2. The source document
3. The user's question to be answered

This prompt template may be refined over multiple rounds of review by comparing the evaluation result.

The cell below applies the draft prompt template to produce the final dataset of:

```json
{
    "prompt": {
        "text": "...",  // Final prompt input to the LLM - after RAG and prompt templating
    },
    "referenceResponse": {
        "text": "..."  // Expected 'ground truth' answer(s) from the model for this question
    }
}
```

This final format is ready to be used with Amazon Bedrock or SageMaker Foundation Model Evaluation jobs.

In [ ]:
prompt_template = Template("""Human:
You are Friendo, a helpful assistant from AnyCompany. Answer the question below based on the
following AnyCompany documentation, or if the answer isn't present in the documentation, you can
say you don't know.

<document>
${doc}
</document>

<question>
${question}
</question>

Assistant:
""")

with open("question-answering/qa.manifest.jsonl") as fin:
    with open("question-answering/eval-job-input-qa.manifest.jsonl", "w") as fout:
        for ix, line in enumerate(fin):
            raw_datum = json.loads(line)
            if ix > 0:
                fout.write("\n")
            # TODO: Use prompt_template.get_identifiers() in Python 3.11+
            # It'd be better to filter the substitute() mapping dict by template vars rather than
            # use safe_substitute (which swallows errors for missing mapping items as well as unused
            # mapping items), but this feature is only available in Python 3.11+
            fout.write(
                json.dumps({
                    "prompt": {"text": prompt_template.safe_substitute(raw_datum)},
                    "referenceResponse": {"text": "<OR>".join(raw_datum["answers"])},
                })
            )

## Using the data with Bedrock and SageMaker

To actually create an Amazon Bedrock or Amazon SageMaker model evaluation job from this manifest file, you'll first need to upload it to Amazon S3:

In [ ]:
# External Dependencies:
import boto3  # AWS SDK for Python
import sagemaker  # Amazon SageMaker high-level SDK

s3 = boto3.resource("s3")
smsess = sagemaker.Session()
bucket_name = smsess.default_bucket()  # (You could specify a custom bucket name instead)
prefix = "llm-eval/demo/squad"

In [ ]:
!aws s3 cp question-answering/eval-job-input-qa.manifest.jsonl s3://{bucket_name}/{prefix}/input.manifest.jsonl

### Review automated analysis results

Once you've created and finished an evaluation job using the above input manifest URI, you can also consider downloading the results to the notebook for analysis:

In [ ]:
# TODO: Replace the results_uri with your job's generated output location!
results_uri = f"s3://{bucket_name}/llm-eval/demo/squad/results/za6h418kkvak/datasets/"
acc_results_uri = f"{results_uri}dataset1/output.jsonl"
acc_results_key = acc_results_uri[len("s3://"):].partition("/")[2]
tox_results_uri = f"{results_uri}dataset2/output.jsonl"
tox_results_key = acc_results_uri[len("s3://"):].partition("/")[2]

os.makedirs("results/squad", exist_ok=True)
s3.Bucket(bucket_name).download_file(acc_results_key, "results/squad/auto-result-acc.output.jsonl")
s3.Bucket(bucket_name).download_file(tox_results_key, "results/squad/auto-result-tox.output.jsonl")

In this example the default Accuracy measure seems to be **penalizing the model** for providing answers that are much more verbose than the reference: Scores are low despite the answers usually seeming correct.

In [11]:
pd.options.display.max_colwidth = 70

auto_acc_records = []
with open("results/squad/auto-result-acc.output.jsonl") as f:
    for line in f:
        datum = json.loads(line)
        auto_acc_records.append({
            "target_output": datum["inputRecord"]["referenceResponse"],
            "model_output": datum["modelInvocations"][0]["responseText"],
            "score_acc": next(
                s for s in datum["automatedEvaluationResult"]["scores"] if s["metricName"] == "Accuracy"
            )["result"],
        })
out_df = pd.DataFrame(auto_acc_records)
out_df

,target_output,model_output,score_acc
0,France,"Based on the documentation provided, Normandy is located in Franc...",0.083333
1,Computational complexity theory,"Based on the documentation, computational complexity theory is th...",0.101695
2,SoCal,"Based on the document, Southern California is often abbreviated a...",0.068966
3,BSkyB,"According to the document, the company formed by the merger of Sk...",0.066667
4,highly diversified<OR>diversified,"Based on the information in the AnyCompany documentation, Victori...",0.049383
5,the southern and central parts of France<OR>about one-eighth<OR>so...,"Based on the passage, the Huguenot population in France was large...",0.324324
6,"solar power, nuclear power or geothermal energy<OR>solar<OR>solar ...","Based on the documentation, solar power is noted as a non-combust...",0.266667
7,8,"Based on the documentation provided, the atomic number of oxygen ...",0.080000
8,1973<OR>October 1973<OR>October,"According to the document, the 1973 oil crisis began in October 1...",0.129032
9,"a body of treaties and legislation, such as Regulations and Direct...",European Union law is a system of laws that apply to the member s...,0.189781
